In [30]:
import math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
import colorsys
import pandas as pd
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import graphviz
from statistics import mean, median, mode, stdev
import scipy.stats as stats

plt.rcParams['figure.constrained_layout.use'] = False
SEED = 7355608

In [31]:
def rubin_combine(vals, val_vars, log_normal=False):
    """Combines values from multiple imputations using Rubin's rules.
    
    Args:
        vals: list of values to combine.
        val_vars: list of within-imputation variances. Note that these should already be in log form if log_normal=True.
        log_normal: Bool, set to True for log-normally distributed values (e.g. odds ratios)
                   and False for normally distributed values
    """
            
    # Pool the values according to Rubin's rules
    if log_normal:
        transformed_vals = [math.log(val) for val in vals]
        mean_val = mean(transformed_vals)
        pooled_val = math.exp(mean_val)
    else:
        mean_val = mean(vals)
        pooled_val = mean_val
    
    # Mean within-imputation variance
    mean_imp_var = mean(val_vars)
    
    # Between-imputation variance
    n_impute = len(vals)
    if log_normal:
        between_imp_var = np.sum((np.array(transformed_vals) - mean_val) ** 2) / (n_impute - 1)
    else:
        between_imp_var = np.sum((np.array(vals) - mean_val) ** 2) / (n_impute - 1)
                                
    # Total variance, combining within and between imputations
    total_var = mean_imp_var + (1 + (1/n_impute))*between_imp_var
                                
    # Degrees of freedom (https://bookdown.org/mwheymans/bookmi/rubins-rules.html)
    r = (between_imp_var + (between_imp_var/n_impute)) / mean_imp_var
    nu = (n_impute - 1) * (1 + 1/r )**2
    
    alpha = 0.05
    t_val = stats.t.ppf(1 - alpha/2, df=nu)  # t-distribution

    if log_normal:
        lower_ci = math.exp(mean_val - t_val * np.sqrt(total_var))
        upper_ci = math.exp(mean_val + t_val * np.sqrt(total_var))
    else:
        lower_ci = mean_val - t_val * np.sqrt(total_var)
        upper_ci = mean_val + t_val * np.sqrt(total_var)

    return pooled_val, lower_ci, upper_ci, vals

In [32]:
'''Generate decision trees for interpretation, using two strategies: 
    1. Fit a decision tree to the entire dataset, concatenating all imputed versions
    2. Fit a decision tree separately to each imputed version of the dataset (focusing on a variable of interest), and examine the distribution
'''

def man_cmap(cmap, value=1.):
    colors = cmap(np.arange(cmap.N))
    hls = np.array([colorsys.rgb_to_hls(*c) for c in colors[:,:3]])
    hls[:,1] *= value
    rgb = np.clip(np.array([colorsys.hls_to_rgb(*c) for c in hls]), 0,1)
    return mcolors.LinearSegmentedColormap.from_list("", rgb)

pt_data = pd.read_csv("miceRanger_imputed_formatted_Brighten-v1_all.csv")

predictors = ['base_phq9', 'gad7_sum', 'age', 'sds_sum', 'alc_sum', 'gender', 'education', 'working', 'marital_status', 'race', 'income_satisfaction', 'mania_history', 'psychosis_history']

for outcome in ["response"]:
    
    print("OUTCOME: " + outcome)
    
    x = pt_data[predictors].copy()
        
    clf_all = tree.DecisionTreeClassifier(max_depth=1)
    clf_all = clf_all.fit(x, pt_data[outcome])
    clf_data = tree.export_graphviz(clf_all, out_file=None, feature_names=predictors, class_names=["no-" + outcome, outcome])
    graph = graphviz.Source(clf_data)
    graph.render("decision_tree_" + outcome)
    
    #http://www.futurile.net/2016/02/27/matplotlib-beautiful-plots-with-style/
    plt.style.use('ggplot')
    txt_col = 'k'
    plt.rcParams['text.color'] = txt_col
    plt.rcParams['axes.labelcolor'] = txt_col
    plt.rcParams['xtick.color'] = txt_col
    plt.rcParams['ytick.color'] = txt_col
    plt.rcParams['axes.labelsize'] = 16
    plt.rcParams['axes.labelweight'] = 'bold'
    plt.rcParams['xtick.labelsize'] = 16
    plt.rcParams['ytick.labelsize'] = 16
    plt.rcParams['legend.fontsize'] = 16
    plt.rcParams['figure.titlesize'] = 24
    plt.tight_layout()

    feature_of_interest = "gad7_sum"
    verbose=False
    thresholds = []
    for imp in pt_data["_Imputation_"].unique():
        pt_data_imp = pt_data[pt_data["_Imputation_"] == imp]

        x = pt_data_imp[[feature_of_interest]].copy()
            
        clf_all = tree.DecisionTreeClassifier(max_depth=1)
        clf_all = clf_all.fit(x, pt_data_imp[outcome])

        # Get feature and threshold used by the tree
        feature = clf_all.tree_.feature[0]  # Get feature index used at root node
        threshold = clf_all.tree_.threshold[0]  # Get threshold value used at root node
        thresholds.append(threshold)

        # Get class distributions for left and right nodes to determine relationship direction
        left_dist = clf_all.tree_.value[1]  # Class distribution when feature <= threshold
        right_dist = clf_all.tree_.value[2]  # Class distribution when feature > threshold
        left_prob = left_dist[0][1] / left_dist[0].sum()  # Probability of response=True when <= threshold
        right_prob = right_dist[0][1] / right_dist[0].sum()  # Probability of response=True when > threshold

        if verbose:
            direction = "more" if right_prob > left_prob else "less"
            print(f"\nThreshold: {feature_of_interest} = {threshold:.2f}")
            print(f"Values above threshold are {direction} likely to have response=True")
    
    print(f"** Statistics for threshold values ({feature_of_interest}) among {pt_data["_Imputation_"].nunique()} imputations: **")
    print(f"Mean threshold: {mean(thresholds)}")
    print(f"Median threshold: {median(thresholds)}")
    print(f"Mode threshold: {mode(thresholds)}")
    print(f"Standard deviation of thresholds: {stdev(thresholds)}")
    print(f"Minimum threshold: {min(thresholds)}")
    print(f"Maximum threshold: {max(thresholds)}")
    print(f"Unique thresholds: {np.unique(thresholds)}")
    print(f"Proportion of thresholds equal to mode={mode(thresholds)}: {np.mean(np.array(thresholds) == mode(thresholds))}")


OUTCOME: response
** Statistics for threshold values (gad7_sum) among 100 imputations: **
Mean threshold: 10.6
Median threshold: 10.5
Mode threshold: 10.5
Standard deviation of thresholds: 0.4380858271151806
Minimum threshold: 10.5
Maximum threshold: 12.5
Unique thresholds: [10.5 12.5]
Proportion of thresholds equal to mode=10.5: 0.95


<Figure size 640x480 with 0 Axes>

In [33]:
'''Generate odds ratios for treatment response (using a single-variable threshold of interest), pooling across imputed versions of the dataset using Rubin's rules. 
'''

n_impute=100


for group in ["all"]:
    print("STUDY GROUP:", group)

    pt_data = pd.read_csv(f"miceRanger_imputed_formatted_Brighten-v1_{group}.csv")

    pt_data = pt_data[["_Imputation_", "gad7_sum", "response"]]

    #pt_data["finish"] = pt_data["bddybocs_tot_recalc_post"].notnull()
    pt_data["gad7_10_or_lower"] = pt_data["gad7_sum"] <= 10
    pt_data["gad7_above_10"] = pt_data["gad7_sum"] > 10

    for cond in ["gad7_10_or_lower", "gad7_above_10"]:
        print("Condition: " + cond)
        for outcome in ["response"]:
            print("----OUTCOME: " + outcome + "----")

            or_vals = []
            or_vars = []
            for imp in pt_data["_Imputation_"].unique():
                pt_data_imp = pt_data[pt_data["_Imputation_"] == imp]
                cont_table = np.array([
                    [len(pt_data_imp[pt_data_imp[cond] & pt_data_imp[outcome]]), len(pt_data_imp[~pt_data_imp[cond] & pt_data_imp[outcome]])],
                    [len(pt_data_imp[pt_data_imp[cond] & ~pt_data_imp[outcome]]), len(pt_data_imp[~pt_data_imp[cond] & ~pt_data_imp[outcome]])]
                ])

                # If any cell of the contingency table is zero, add 0.5 to all cells. 
                if np.any(cont_table == 0):
                    cont_table = cont_table + 0.5

                # Within-imputation value estimate
                or_vals.append((cont_table[0,0]/cont_table[0,1]) / (cont_table[1,0]/cont_table[1,1]))

                # Within-imputation variance
                or_vars.append(sum([1/val for val in cont_table.reshape(-1)]))

            pooled_or, lower_ci, upper_ci, ors = rubin_combine(or_vals, or_vars, log_normal=True)
                                                            
            print("RUBIN OddsR:", round(pooled_or, 3), "[", round(lower_ci, 3), ",", round(upper_ci, 3), "].")
            print("avg OddsR:", round(sum(ors)/len(ors), 3))
            print("based on ", len(ors), " odds ratios")
            
        print("---------")
    print("==============================")

STUDY GROUP: all
Condition: gad7_10_or_lower
----OUTCOME: response----
RUBIN OddsR: 3.135 [ 2.06 , 4.77 ].
avg OddsR: 3.148
based on  100  odds ratios
---------
Condition: gad7_above_10
----OUTCOME: response----
RUBIN OddsR: 0.319 [ 0.21 , 0.485 ].
avg OddsR: 0.32
based on  100  odds ratios
---------


In [34]:
import pandas as pd
import numpy as np
from scipy import stats

def analyze_gad7_sds_correlation(df):
    """
    Calculate correlation between GAD-7 and SDS scores, handling missing values.
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing 'gad7_sum' and 'sds_sum' columns
    
    Returns:
    dict: Dictionary containing correlation statistics and sample sizes
    """
    # Remove rows where either GAD-7 or SDS is missing
    complete_cases = df.dropna(subset=['gad7_sum', 'sds_sum'])
    
    # Calculate number of complete and missing cases
    n_complete = len(complete_cases)
    n_total = len(df)
    n_missing_gad7 = df['gad7_sum'].isna().sum()
    n_missing_sds = df['sds_sum'].isna().sum()
    
    # Calculate correlations if we have at least 2 complete cases
    if n_complete >= 2:
        pearson_r, pearson_p = stats.pearsonr(
            complete_cases['gad7_sum'], 
            complete_cases['sds_sum']
        )
        spearman_r, spearman_p = stats.spearmanr(
            complete_cases['gad7_sum'], 
            complete_cases['sds_sum']
        )
    else:
        pearson_r = pearson_p = spearman_r = spearman_p = np.nan
    
    results = {
        'n_total': n_total,
        'n_complete': n_complete,
        'n_missing_gad7': n_missing_gad7,
        'n_missing_sds': n_missing_sds,
        'pearson_r': pearson_r,
        'pearson_p': pearson_p,
        'spearman_r': spearman_r,
        'spearman_p': spearman_p
    }

    # Print results
    print(f"Complete cases: {results['n_complete']} out of {results['n_total']}")
    print(f"Missing GAD-7: {results['n_missing_gad7']}")
    print(f"Missing SDS: {results['n_missing_sds']}")
    print(f"Pearson correlation: r = {results['pearson_r']:.3f}, p = {results['pearson_p']:.3f}")
    print(f"Spearman correlation: ρ = {results['spearman_r']:.3f}, p = {results['spearman_p']:.3f}")

    return results

In [35]:
pt_data = pd.read_csv(f"miceRanger_imputed_formatted_Brighten-v1_{group}.csv")

results = analyze_gad7_sds_correlation(pt_data)

Complete cases: 49300 out of 49300
Missing GAD-7: 0
Missing SDS: 0
Pearson correlation: r = 0.553, p = 0.000
Spearman correlation: ρ = 0.556, p = 0.000


In [36]:
pt_data_missing = pd.read_csv(f"missing_formatted_Brighten-v1_{group}.csv")

results = analyze_gad7_sds_correlation(pt_data_missing)

Complete cases: 474 out of 493
Missing GAD-7: 18
Missing SDS: 16
Pearson correlation: r = 0.546, p = 0.000
Spearman correlation: ρ = 0.547, p = 0.000
